# COVID-19 Testing Episodes
Written by: Branson Chen, Hannah Chung, Kinwah Fung <br>
Last modified: 20210225

## Table of Contents

<a href='#Overview'>Overview</a><br>
<a href='#Input-variables'>Input variables</a><br>
<a href='#Importing-data'>Importing data</a><br>
<a href='#Flag-different-test-types'>Flag different test types</a><br>
<a href='#Roll-up-to-testing-episode'>Rollup to testing episode</a><br>
<a href='#Final-output'>Final output</a><br>
- <a href='#Data-definitions'>Data definitions</a><br>
- <a href='#Additional-information'>Additional information</a><br>

## Overview

- This script takes as input the file created by COVID19_processing.ipynb
    - Please specify output_flag = 1 or 2 in the previous program so the dataset contains the required variables
- Next, exclusions are applied to remove observations with resultstatus = N/X/W and observations with blank covid variable
- Some variables are created or assigned (e.g., interpretation_flag, covidtest, testtype)
- The roll-up from TEST RESULTS to TESTING EPISODES occurs with multiple steps, for each test type:
    - TEST RESULTS are rolled-up to TEST REQUESTS: for each TEST REQUEST (ordersid+fillerordernumberid), select the result by prioritizing latest release time and then clear covid results (covidtest) and then interpretations (interpretation_flag) and then covid result hierarchy (covidcode; P>I>N>D>C>R) 
    - TEST REQUESTS are rolled-up to LAB ORDERS: for each LAB ORDER (ordersid), select the result by prioritizing clear covid results (covidtest) and then latest release time and then covid result hierarchy (covidcode; P>I>N>D>C>R) 
    - LAB ORDERS (with a patientid) are rolled-up to TESTING EPISODES: for each TESTING EPISODE (patientid+observationdate), select the result by prioritizing the covid result hierarchy (covidcode; P>I>N>D>C>R) and then latest release time and then lowest ordersid
- Multiple lab orders for each testing episode will be transposed and added to the final dataset (ordersid1-n)
- Multiple lab orders that contain the final covid result for each testing episode will be concatenated and added to the final dataset (final_result_ordersids)
- Records without patientids will then be appended to the final dataset

## Input variables

In [ ]:
#input path and filename
input_path = ''
input_filename = 'output.csv'

#output filename #1
output_filename1 = 'episodes'

#output filename #2
output_filename2 = 'laborders'

#output filename #3
output_filename3 = 'testrequests'

#output additional datasets (list the datasets to output)
#1 = episode-level, 2 = laborder-level, 3 = testrequest-level
output_list = [1,]

## Importing data

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df_raw = pd.read_csv(input_path+input_filename, dtype={'patientid':np.object}, low_memory=False)

In [ ]:
#keep necessary columns
df = df_raw.copy(deep = True)
df = df[['patientid', 'ordersid', 'fillerordernumberid', 'observationdatetime', 'testrequestcode',
         'observationcode', 'observationreleasets', 'observationresultstatus', 
         'exclude_flag', 'covid']]
df = df.fillna('')

df['ordersid'] = df['ordersid'].apply(int)
df['observationdatetime'] = pd.to_datetime(df['observationdatetime'])
df['observationreleasets'] = pd.to_datetime(df['observationreleasets'])
print('Number of records from original dataset: ' + str(len(df)))

In [ ]:
#remove observations based on observationresultstatus
print('Number of records removed with result status N/X: '+str(sum(df['observationresultstatus'].isin(('N','X')))))
df_clean = df[~df['observationresultstatus'].isin(('N','X'))]

print('Number of records removed with result status W (exclude_flag): '+str(sum(df_clean['exclude_flag'] == 'Y')))
df_clean = df_clean[df_clean['exclude_flag'] == 'N']

print('Number of records remaining: '+str(len(df_clean)))
print('Breakdown of covid variable:')
print(df_clean['covid'].value_counts())

In [ ]:
#remove records with blank covid variable
print('Number of records removed with blank covid variable: '+str(sum(df_clean['covid'] == '')))
df_clean = df_clean[df_clean['covid'] != '']

print('Number of records remaining: '+str(len(df_clean)))

In [ ]:
#hierarchy: P > I > N > D > C > R 
#P = Positive, I = Indeterminate, N = Negative, D = penDing, C = Cancelled, R = Rejected/invalid
result_mappings = {'P':1,'I':2,'N':3,'D':4,'C':5,'R':6}

#assign S (presumptive-positive) as P (positive)
df_clean.loc[df_clean['covid'] == 'S', 'covid'] = 'P'

#convert covid result variable (from previous script) to number for hierarchy
df_clean['covidcode'] = df_clean['covid'].map(result_mappings)

###create new variables
#interpretation_flag: observations that have a covid interpretation code
df_clean['interpretation_flag'] = df_clean['observationcode'].apply(
    lambda x: 'T' if x in ('XON10842-3','XON12338-0','XON13527-7') else 'F')
#covidtest: observations that have a clear covid result (P, I, N, D)
df_clean['covidtest'] = df_clean['covidcode'].apply(lambda x: 'T' if x in (1,2,3,4) else 'F')
#date versions of datetime
df_clean['observationdate'] = df_clean['observationdatetime'].apply(lambda x: np.datetime64(x, 'D'))
df_clean['observationreleasedate'] = df_clean['observationreleasets'].apply(lambda x: np.datetime64(x, 'D'))

#drop used columns
df_clean.drop(['observationresultstatus','exclude_flag','observationcode','observationdatetime'], 1, inplace=True)

## Flag different test types

In [ ]:
#set all records to blank testtype (standard pcr) as default
df_clean['testtype'] = ''

#set rapid tests based on testrequestcode = TR12946-0
df_clean.loc[df_clean['testrequestcode'].isin(['TR12946-0',]), 'testtype'] = '_rapid'

#set rapid tests based on testrequestcode = TR12947-8
df_clean.loc[df_clean['testrequestcode'].isin(['TR12947-8',]), 'testtype'] = '_rapid_a'

#covid observations by testtype
df_clean['testtype'].value_counts()

## Roll-up to testing episode

In [ ]:
###ROLL UP from TEST RESULTS to TEST REQUESTS
#hierarchy: latest observationreleasets > covidtest = 'T' > interpretation_flag = 'T' > covidcode (P>I>N>D>C>R)
df_clean.sort_values(
        ['ordersid','fillerordernumberid','observationreleasets','covidtest','interpretation_flag','covidcode'],
        ascending=[True, True, False, False, False, True], 
        inplace=True)

def testresults_to_testrequests(testtype):
    #separate by testtype
    df_testrequests = df_clean.loc[df_clean['testtype'] == testtype, :].drop(['testtype','testrequestcode'],1).groupby(
        ['ordersid','fillerordernumberid']).first().reset_index()
    
    print('Number of TEST REQUESTS: '+str(len(df_testrequests)))
    return df_testrequests

In [ ]:
###ROLL UP from TEST REQUESTS to LAB ORDERS
#hierarchy: covidtest = 'T' > latest observationreleasets > covidcode (P>I>N>D>C>R)
def testrequests_to_laborders():
    df_testrequests.sort_values(['ordersid','covidtest','observationreleasets','covidcode'],
                                ascending=[True, False, False, True],
                                inplace=True)

    df_laborders = df_testrequests.drop(['fillerordernumberid','interpretation_flag'],1)\
                                    .groupby(['ordersid']).first().reset_index()
    
    print('Number of LAB ORDERS: '+str(len(df_laborders)))
    return df_laborders

In [ ]:
#split up records without a patientid
def split_patientid():
    df_laborders_nopat = df_laborders[df_laborders['patientid'] == ''].copy()
    df_laborders_pat = df_laborders[df_laborders['patientid'] != ''].copy() 
    
    print('Number of LAB ORDERS without a patientid: ' + str(len(df_laborders_nopat)))
    print('Number of LAB ORDERS with a patientid: ' + str(len(df_laborders_pat)))
    return df_laborders_nopat, df_laborders_pat

In [ ]:
###ROLL UP from lab orders to testing episodes (distinct by patientid-observationdate)
#hierarchy: covidcode (P>I>N>D>C>R) > latest observationreleasets > lowest ordersid
def laborders_to_testingepisodes():
    df_laborders_pat.sort_values(['patientid','observationdate','covidcode','observationreleasets','ordersid'],
                                 ascending=[True, True, True, False, True], 
                                 inplace=True)

    df_episodes = df_laborders_pat.groupby(['patientid','observationdate']).first().reset_index()
    
    print('Number of TESTING EPISODES (with a patientid): '+str(len(df_episodes)))
    return df_episodes

In [ ]:
#transpose all orders for each episode
def transpose_orders(testtype):
    df_pivot = df_laborders_pat[['patientid','observationdate','ordersid']].copy()
    df_pivot.sort_values(['patientid','observationdate','ordersid'], inplace=True)

    df_pivot['order_num'] = df_pivot.groupby(['patientid','observationdate'])['ordersid'].rank(method='first')
    df_pivot['order_num'] = df_pivot['order_num'].apply(lambda x: 'ordersid' + str(int(x)))
    df_pivot = pd.pivot_table(df_pivot, 
                              values='ordersid', 
                              index=['patientid','observationdate'], 
                              columns='order_num').reset_index()

    #new variable that counts number of pivoted ordersids
    df_pivot['numordersid'] = df_pivot.iloc[:,2:].count(axis=1)
    
    #use pcr df_pivot as base structure assuming it has the most ordersids in one testing episode
    if testtype != '':
        df_pivot = pd.concat([df_pivot_base, df_pivot], axis=0, sort=False)
        
    print('Most ordersids in one TESTING EPISODE:', max(df_pivot['numordersid']))
    
    return df_pivot, df_pivot[0:0]

In [ ]:
#concatenate ordersids with the same FINAL covidcode for each testing episode into final_result_ordersids
def final_results():
    df_result_ordersids = pd.merge(df_episodes[['patientid','observationdate','covidcode']],
                                   df_laborders_pat[['patientid','observationdate','covidcode',
                                                     'observationreleasets','ordersid']],
                                   on=['patientid','observationdate','covidcode'],
                                   how='inner')
    df_result_ordersids.sort_values(['patientid','observationdate','covidcode','observationreleasets','ordersid'],
                                    ascending=[True,True,True,False,True],
                                    inplace=True)
    
    df_result_ordersids = df_result_ordersids.groupby(['patientid','observationdate','covidcode'])\
        ['ordersid'].apply(lambda x: ','.join(map(str,x))).reset_index()
    df_result_ordersids = df_result_ordersids.rename(columns={'ordersid':'final_result_ordersids'}).drop('covidcode',1)
    
    return df_result_ordersids

In [ ]:
#merging testing episodes with pivoted orders and final_result_ordersids
def final_dataset():
    df_final = pd.merge(df_pivot, df_episodes.drop(['ordersid','covidcode','observationreleasets'],1),
                        on=['patientid','observationdate'], how='inner')
    df_final = pd.merge(df_final, df_result_ordersids, on=['patientid','observationdate'], how='inner')

    #adding the records with no patient id
    df_laborders_nopat.rename(columns={'ordersid':'ordersid1'}, inplace=True)
    df_laborders_nopat['final_result_ordersids'] = df_laborders_nopat['ordersid1'].apply(str)
    df_laborders_nopat['numordersid'] = 1
    df_final = pd.concat([df_final,
                         df_laborders_nopat.drop(['covidcode','observationreleasets'],1)],
                         sort=False).rename(columns={'covid':'covidresult'})
    df_final = df_final.sort_values(['patientid','observationdate'])

    #reorder the columns to have ordersid1-10 in order and ordersid11+ at the end
    ordersids_cols = [o for o in df_final.columns if o.startswith('ordersid')]
    ordersids_cols_s = sorted(ordersids_cols, key=lambda x: int(''.join(filter(lambda y: y.isdigit(), x))))

    reordered_cols = ['patientid','observationdate'] + ordersids_cols_s[0:10] + ['numordersid', 'covidresult', 'covidtest',
           'observationreleasedate', 'final_result_ordersids'] + ordersids_cols_s[10:]
    
    df_final = df_final.loc[:,reordered_cols]
    
    print('Number of TESTING EPISODES (including records without patientids): ' + str(len(df_final)))
    print('Breakdown of covid variable:')
    print(df_final['covidresult'].value_counts())
    return df_final

## Final output

In [ ]:
#go through all of the test types in df_clean
for testtype in sorted(df_clean['testtype'].unique()):
    print('------- PROCESSING TEST TYPE: ***'+('pcr' if testtype == '' else testtype[1:])+'*** -------')
    df_testrequests = testresults_to_testrequests(testtype)
    df_laborders = testrequests_to_laborders()
    df_laborders_nopat, df_laborders_pat = split_patientid()
    df_episodes = laborders_to_testingepisodes()
    df_pivot, df_pivot_base = transpose_orders(testtype)
    df_result_ordersids = final_results()
    df_final = final_dataset()
    
    #FINAL RESULT TO OUTPUT
    for i in output_list:
        if i == 1:
            df_final.to_csv(output_filename1+testtype+'.csv', index=False)
        if i == 2:
            df_laborders.to_csv(output_filename2+testtype+'.csv', index=False)
        if i == 3:
            df_testrequests.to_csv(output_filename3+testtype+'.csv', index=False)
        
print('------- COMPLETE -------')


### Data definitions

- patientid: PATIENTID variable from input file
- observationdate: Specimen collection date
- ordersid1-n: Transposed ORDERSIDs for this OBSERVATIONDATE
- numordersid: Total # ORDERSIDs for this OBSERVATIONDATE [specimen collection date]
- covidresult: COVID19 test result using hierarchy (Positive > Indeterminate > Negative > penDing > Cancelled > Rejected)
- covidtest: Is it a COVID test (T/F)? T if COVIDRESULT = Positive, Indeterminate, Negative, or penDing
- observationreleasedate: Observation release date
- final_result_ordersids: Comma-delimited ORDERSIDs that have the same final COVIDRESULT after rolling-up

### Additional information

- If there are multiple ordersids that have the same final covidresult in the episode, they are listed in final_result_orderids, ordered by the latest observationrelesets then by lowest ordersid
- The observationreleasedate variable comes from the first ordersid listed in final_result_orderids (the first listed has the **latest** observationreleasets)
- We suggest using the first ordersid in final_result_orderids to retrieve any additional order-level information from the original data source